## 6. 한글텍스트 - 분류 모델링
### 로지스틱 회귀 모델 with TF-IDF

### 라이브러리 불러오기

In [1]:
import os
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [2]:
# 경고메세지 무시
import warnings
warnings.filterwarnings(action='ignore') 

### 데이타 불러오기 - 텍스트 형태

In [3]:
DATA_IN_PATH = './data_in/' 
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'nsmc_train_clean.csv'
TEST_CLEAN_DATA = 'nsmc_test_clean.csv'

RANDOM_SEED = 42
TEST_SPLIT = 0.2

In [4]:
train_df = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA )
test_df = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA )

### 결측치 확인

In [7]:
train_df.isna().sum()

id             0
document    1282
label          0
dtype: int64

In [9]:
train_df.isna().sum().sum()

1282

In [8]:
test_df.isna().sum()

id            0
document    436
label         0
dtype: int64

In [10]:
# 결측치 제거
train_df = train_df.dropna()
test_df = test_df.dropna()

In [13]:
train_df.shape

(148718, 3)

In [11]:
test_df.shape

(49564, 3)

### TF-IDF 벡터화
- min_df : 설정한 값보다 특정 토큰의 df 값이 적게 나오면 벡터화 과정에서 제거한다는 의미
- max_df : 설정한 값보다 특정 토큰의 df 값이 크게 나오면 벡터화 과정에서 제거한다는 의미
- ngram_range : 빈도의 기본 단위를 어느 범위의 n-gram으로 설정할 것인지를 지정하는 인자

#### 형태소 분석기 함수화

In [14]:
from konlpy.tag import Okt

okt = Okt()
def okt_tokenizer(text):
    # 입력 인자로 들어온 text 를 형태소 단어로 토큰화 하여 list 객체 반환
    tokens_ko = okt.morphs(text)
    return tokens_ko

In [15]:
okt_tokenizer("꼭 대박나세요")

['꼭', '대박나세요']

#### TF-IDF 피처벡터화

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Okt 객체의 morphs( ) 객체를 이용한 tokenizer를 사용. ngram_range는 (1,2) 
tfidf_vect = TfidfVectorizer(tokenizer=okt_tokenizer, ngram_range=(1,2), min_df=3, max_df=0.9)
tfidf_vect.fit(train_df['document'])
tfidf_matrix_train = tfidf_vect.transform(train_df['document'])

#### DTM (단어사전행렬) 확인

In [17]:
print(tfidf_vect.vocabulary_)

{'더빙': 21641, '진짜': 85433, '짜증나다': 86259, '목소리': 33943, '더빙 진짜': 21675, '진짜 짜증나다': 85824, '짜증나다 목소리': 86279, '흠': 98759, '포스터': 92674, '보고': 38622, '초딩': 87968, '영화': 61699, '줄': 83609, '오버': 63775, '연기': 60753, '조차': 81495, '가볍다': 1334, '않다': 53217, '포스터 보고': 92691, '초딩 영화': 87983, '영화 줄': 62717, '오버 연기': 63782, '연기 조차': 60969, '가볍다 않다': 1346, '너': 15528, '무재': 34770, '다그': 19306, '래서': 27120, '보다': 38997, '추천': 88698, '다': 18309, '너 무재': 15552, '래서 보다': 27121, '보다 추천': 40285, '교도소': 7649, '이야기': 72143, '구먼': 7800, '솔직하다': 46971, '재미': 77063, '없다': 56341, '평점': 92340, '조정': 81473, '솔직하다 재미': 47030, '재미 없다': 77135, '없다 평점': 57021, '평점 조정': 92550, '사이': 43916, '그': 8394, '익살스럽다': 72893, '돋보이다': 23211, '스파이더맨': 48393, '에서': 59051, '늙다': 17983, '보이': 40679, '하다': 93870, '커스틴': 89548, '너무나도': 16026, '이쁘다': 71826, '연기 돋보이다': 60822, '돋보이다 영화': 23215, '늙다 보이': 17990, '보이 다': 40684, '다 하다': 19244, '이쁘다 보이': 71854, '막': 30008, '걸음': 4670, '마': 29476, '떼다': 26107, '세': 46137, '부터': 41474, '초등학교'

In [18]:
tfidf_matrix_train 

<148718x99106 sparse matrix of type '<class 'numpy.float64'>'
	with 2569853 stored elements in Compressed Sparse Row format>

In [19]:
sentence = [train_df['document'][0]] 
print(tfidf_vect.transform(sentence).toarray())

[[0. 0. 0. ... 0. 0. 0.]]


### 로지스틱 회귀 모델 분류

In [20]:
from sklearn.linear_model import LogisticRegression
# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
# 1) 모델 생성
lg_clf = LogisticRegression(random_state=0)

In [21]:
lg_clf 

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [23]:
# 2) 모델 학습 - model.fit(x_train, y_train)
lg_clf.fit(tfidf_matrix_train , train_df['label'] )

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 테스트 데이타를 이용한 정확도 평가

### 1) 테스트 데이타를 TD-IDF로 피처 벡터화 
- 주의 : 학습데이타의 형태대로 피처 벡터화

In [24]:
# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

### 2) 테스트 데이타를 이용한 예측 및 평가

In [25]:
from sklearn.metrics import accuracy_score

# 3) 테스트 데이타 예측 - model.predict(x_test)
preds = lg_clf.predict(tfidf_matrix_test)

# 4) 정확도 평가 - model.score(y_test, preds)
print('Logistic Regression 정확도: ', accuracy_score(test_df['label'], preds))

Logistic Regression 정확도:  0.8625010087967073


In [26]:
# 예측값
pred = lg_clf.predict(tfidf_matrix_test)

In [27]:
# 실제값
test_df['label']

0        1
2        0
3        0
4        0
5        1
        ..
49995    1
49996    0
49997    0
49998    0
49999    0
Name: label, Length: 49564, dtype: int64

In [28]:
# 정확도
sum(pred == test_df['label'] )

42749

In [29]:
42749/49564

0.8625010087967073

### 로지스틱 회귀 모델 분류 & GridSearchCV

### 1) 3-교차검증

In [37]:
from sklearn.model_selection import cross_val_score , cross_validate

# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 3개 
scores = cross_val_score(lg_clf, tfidf_matrix_train, train_df['label'], scoring='accuracy',cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

교차 검증별 정확도: [0.8605 0.856  0.857 ]
평균 검증 정확도: 0.8579


### 2) 교차검증과 최적 하이퍼파라미터 튜닝을 한번에 하기
- GridSearchCV  사용

In [32]:
from sklearn.model_selection import GridSearchCV

# Logistic Regression 을 이용하여 감성 분석 Classification 수행. 
lg_clf = LogisticRegression(random_state=0)

# Parameter C 최적화를 위해 GridSearchCV 를 이용. 
params = { 'C': [1 ,3.5, 4.5, 5.5, 10 ] }
grid_cv = GridSearchCV(lg_clf , param_grid=params , cv=3, scoring='accuracy', verbose=1)
grid_cv.fit(tfidf_matrix_train , train_df['label'] )
print(grid_cv.best_params_ , round(grid_cv.best_score_,4))

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  1.3min finished


{'C': 4.5} 0.8612


In [33]:
grid_cv.best_estimator_

LogisticRegression(C=4.5, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=0, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

### 테스트 데이타에 TF-IDF 벡터 변환 후 정확도 평가

In [34]:
from sklearn.metrics import accuracy_score

# 학습 데이터를 적용한 TfidfVectorizer를 이용하여 테스트 데이터를 TF-IDF 값으로 Feature 변환함. 
tfidf_matrix_test = tfidf_vect.transform(test_df['document'])

# classifier 는 GridSearchCV에서 최적 파라미터로 학습된 classifier를 그대로 이용
best_estimator = grid_cv.best_estimator_
preds = best_estimator.predict(tfidf_matrix_test)

print('Logistic Regression 정확도: ',accuracy_score(test_df['label'],preds))

Logistic Regression 정확도:  0.8650835283673635


## 과제

## 로지스틱 회귀모델 with Word2Vec

## 랜덤포레스트 모델 with TF-IDF

## 랜덤포레스트 모델 with Word2Vec